### ***Importando as bibliotecas***

In [1]:
import torch
import gc
from datasets import load_dataset
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer
from transformers import TrainingArguments

Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### ***Inicializando o Foundation Model***

O Foundation Model escolhido para o treinamento será o **unsloth/tinyllama-bnb-4bit** por conta do tamanho e a utilização da memória. O max_seq_length será utilizado se baseando na quantidade de tokens que retornou na etapa 1.

In [2]:
max_seq_length = 256 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

Foi criado uma função para iniciar o modelo, essa função vai ajudar a limpar a memória caso for necessário carregar o modelo novamente.

In [3]:
# Inicia o modelo e o tokenizer
def iniciar_modelo():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/tinyllama-bnb-4bit",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    
    model = FastLanguageModel.get_peft_model(
        model,
        r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj",],
        lora_alpha = 16,
        lora_dropout = 0, # Supports any, but = 0 is optimized
        bias = "none",    # Supports any, but = "none" is optimized
        # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
        use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
        random_state = 3407,
        use_rslora = False,  # We support rank stabilized LoRA
        loftq_config = None, # And LoftQ
    )
    
    return model, tokenizer

### ***Carregando o dataset***

Como será apenas a validação dos parâmetros de treinamento vou carregar a parte 1 do meu dataset.

In [4]:
# Caminho do arquivo JSON do dataset de treino
caminho_arquivo = './LF-Amazon-1.3M/treino/trn_formatado_parte_1.json'

# Carregar o dataset diretamente do arquivo JSON
dataset_treino = load_dataset("json", data_files=caminho_arquivo, split="train")

# Verificar o dataset carregado
print(dataset_treino[:5])

{'uid': ['0385753845', '1603201599', '3836527863', '1883011868', 'B000BQWPTM'], 'title': ['How to Babysit a Grandma', 'The Mayo Clinic Book of Home Remedies: What to Do For The Most Common Health Problems', 'Guts and Glory: The Golden Age of American Football', 'Tennessee Williams: Plays 1937-1955 (Library of America)', 'NGK (6726) BPMR6A Standard Spark Plug, Pack of 1'], 'content': ["PreS-Gr 2&#x2014;In a companion to Reagan's How to Babysit a Grandpa (Knopf, 2012), a young girl heads over to her grandma's house for a sleepover babysitting session-with the child providing clear and humorous instructions to readers on how to care for a grandma. The to-do list contains many choices for Grandma to select from, including a walk to the park, reading, taking photos, playing dress-up, and adding sugary sprinkles to her meal items. The child wisely allows plenty of time for Grandma to look at the pages while reading a book, peek at the stars, and choose the best spot to sleep. Any grown-up wh

Carregando o dataset que será utilizado para avaliar o modelo depois do treinamento.

In [5]:
# Caminho do arquivo JSON do dataset de teste
caminho_arquivo = './LF-Amazon-1.3M/teste/tst_formatado.json'

# Carregar o dataset diretamente do arquivo JSON
dataset_teste = load_dataset("json", data_files=caminho_arquivo, split="train")

print(dataset_teste[:5])

{'uid': ['B003YI166W', 'B0014DU2QY', 'B00523JL5K', 'B000AOEPMK', '0764599100'], 'title': ['Dreambaby Refrigerator Latch', 'Pacon Classroom Select Pacon Sentence Strip, 3 X 24 in, 1-1/2 in Ruling, 3/4 in Dotted Midline, 1 in Descender Space, Assorted Colors, Pack of 100', 'HTC EVO 3D Accessory - Blossoming Purple Violet Flower Protective Hard Case Cover Design for Sprint 4G', 'The Martha Stewart Holiday Collection - Homemade Holidays', 'Card Games For Dummies'], 'content': ['The Dream Baby Refrigerator Latch helps prevent children from opening fridges or freezers.', 'Pacon Classroom Select Assorted Super Bright Sentence Strip measuring 3 in x 24 in contains 1-1/2 in blue ruling, 3/4 in dotted midline and 1 in descender space. Strip is sold as 100 per pack.', 'Protect your cell phone with our premium Crystal Image case. This accessory provides protection for your cell phone from unnecessary scratch, dent or chips. It is made to fit perfectly and give the phone maximum protection. The har

### ***Fine Tuning***

Foi criado uma função para treinar o modelo, a função consiste nos seguintes passos:

**1.** Limpeza de memória.

- É realizado uma limpeza na memória para o modelo que será carregado.

**2.** Inicialização do modelo.

- Utilizando a função criada acima é realizado a inicialização do modelo e tokenizer.

**3.** Criação do *trainer*.

- É feito a criação do *trainer* passando os parâmetros do modelo e tokenizer criados, dataset de treino e avaliação, o campo que será utilizado para treino, o *max_seq_length* setado no inicio, o *dataset_num_proc* de 8 e os argumentos de treinamento.
- Foi utilizado um *dataset_num_proc* de 8 por ser o valor mais alto que minha CPU aguentou, com isso fazendo um processamento mais rápido do dataset antes de treinar.

**4.** Validação de memória antes e depois do treinamento.

- Essa validação foi utlilizada para que eu tivesse uma noção do quanto de memória o meu treinamento estava utilizando de acordo com os parâmetros passados.

**5.** Realização do treinamento.

- Inicia o treinamento do modelo com o dataset que foi passado.

**6.** Realização da avaliação do modelo.

- Após o treinamento eu realizo uma avaliação do modelo para ter uma ideia de como ficou o desempenho para os dados não vistos durante o treinamento.

**7.** Salvar o modelo.
- Por último é realizado a gravação do modelo em uma pasta.

In [6]:
# Função que treina o modelo
def treinar_modelo(trainingArguments):
    # Limpar a memória da GPU
    torch.cuda.empty_cache()
    gc.collect()
    
    # Inicialização do modelo e tokenizer
    model, tokenizer = iniciar_modelo()
    
    # Criar o SFTTrainer
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset_treino,
        eval_dataset=dataset_teste,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=8,  # Multiprocessamento
        packing=False,  # Se necessário para sequências curtas
        args=trainingArguments
    )
    
    # Verificar a memória da GPU antes do treinamento
    gpu_stats = torch.cuda.get_device_properties(0)
    start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
    print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
    print(f"{start_gpu_memory} GB of memory reserved.")
    
    # Iniciar o treinamento
    trainer_stats = trainer.train()
    
    # Verificar a memória da GPU após o treinamento
    used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
    used_percentage = round(used_memory / max_memory * 100, 3)
    lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
    print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
    print(f"{round(trainer_stats.metrics['train_runtime'] / 60, 2)} minutes used for training.")
    print(f"Peak reserved memory = {used_memory} GB.")
    print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
    print(f"Peak reserved memory % of max memory = {used_percentage} %.")
    print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")
    
    # Avaliar o modelo no dataset de avaliação
    eval_results = trainer.evaluate()

    # Exibir os resultados da avaliação
    print("Resultados da Avaliação:", eval_results)
    
    # Salvar o modelo treinado
    trainer.save_model('./model/lora_model')
    tokenizer.save_pretrained('./model/lora_model')
    print("Modelo salvo com sucesso!")

**Argumentos de treinamento**

Durante alguns dias fui realizando testes relacionados aos parâmetros:
- *per_device_train_batch_size*
- *gradiente_accumultation_steps*
- *warmup_steps*
- *learning_rate*
- *weight_decay*

Para poder encontrar um balanceamento entre:
- **Tempo**
- **Uso de GPU**
- **Resultado Avaliação**

Inicialmente utilizei o max_steps = 60 até encontrar este equilibrio e quando encontrei os parâmetros que me agradaram, que foram os que estão abaixo, rodei 2 épocas para ter certeza de como ficou.

In [7]:
# Argumentos de treinamento
trainingArguments = TrainingArguments(
    per_device_train_batch_size=128,
    gradient_accumulation_steps=2,
    warmup_steps=5,
    num_train_epochs=2,
    #max_steps=60,
    learning_rate=3e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=100,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs"
)

**Treinamento**

Utilizado para realizar o teste a cada parâmetro do *trainingArguments* alterado.

In [8]:
treinar_modelo(trainingArguments)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


GPU = NVIDIA GeForce RTX 4070 Ti. Max memory = 11.994 GB.
0.787 GB of memory reserved.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 139,041 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 128 | Gradient Accumulation steps = 2
\        /    Total batch size = 256 | Total steps = 1,086
 "-____-"     Number of trainable parameters = 12,615,680


Step,Training Loss
100,1.726900
200,1.575000
300,1.557400
400,1.547600
500,1.536300
600,1.515800
700,1.506000
800,1.497600
900,1.494200
1000,1.490600


11568.8032 seconds used for training.
192.81 minutes used for training.
Peak reserved memory = 7.35 GB.
Peak reserved memory for training = 6.563 GB.
Peak reserved memory % of max memory = 61.281 %.
Peak reserved memory for training % of max memory = 54.719 %.


Resultados da Avaliação: {'eval_loss': 1.5033295154571533, 'eval_runtime': 854.2072, 'eval_samples_per_second': 70.21, 'eval_steps_per_second': 8.777, 'epoch': 1.9981600735970562}
Modelo salvo com sucesso!
